In [1]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import pandas as pd
from openpyxl import load_workbook
import re
import io
import os
import pdb

import sys
sys.path.append(os.path.abspath('../kev/'))

import eq_data
import eq_preproc
import eq_evaluator
import eq_postproc
import eq_writer


# basic input ------------------------------------------------
 
def cl_scripts_load(_sep, _subdir, _file):
    
    # if specific file selected it should be XLSX one
    if _file != "":
        
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        _file = _subdir + _file
        
        # open excel file
        with open(_file, "rb") as f:
            inmemory_file = io.BytesIO(f.read())
        wb = load_workbook(inmemory_file, read_only = True)
                
        r = re.compile(r'heats')
        heats_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], index_col = 0)
        
        r = re.compile(r'targets')
        targets = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = None)
        
        r = re.compile(r'enthalpies')
        delta_H = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
    # use a bunch of plain text files instead
    else:
          
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        file_names = list(os.listdir(path = _subdir))

        r = re.compile(r'heats')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        heats_data = pd.read_csv(file, sep = _sep, decimal = ",", index_col = 0)
        
        r = re.compile(r'targets')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        targets = pd.read_csv(file, sep = _sep, header = None)
        
        r = re.compile(r'enthalpies')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        delta_H = pd.read_csv(file, sep = _sep)
        
    return heats_data, targets, delta_H

In [2]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np
import re

# basic preprocessing ----------------------------------------
    
def cl_preproc(con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type):
    
    # checking if there are several series
    
    if 'series' not in con_data.columns:        
        con_data['series'] = ''
    
    if 'series' not in heats_data.index:
        heats_data.loc['series'] = '' # probably not needed
    
    if 'volumes' not in heats_data.index:
        heats_data.loc['volumes'] = 1        

    # series variables
    
    ser_info = con_data['series'].to_numpy()
    ser_unique = np.unique(ser_info)
    ser_num = np.shape(np.unique(ser_info))[0]
    
    ser_counts = con_data.groupby(['series']).size().tolist();
    
    # reading volumes from experimental data
    volumes = heats_data.loc['volumes'].to_numpy()
    
    # reading exp heats from experimental data
    heats = heats_data.loc['observation'].to_numpy() - heats_data.loc['dilution'].to_numpy()
        
    # transforming deviation to the matrix form
    devs = heats_data.loc['deviation'].to_numpy()
    devs = np.diag((1 / (np.array(devs) * np.array(devs))) * (np.sum(np.array(devs) * np.array(devs)) / len(devs)))
    
    # creating vector of known enthalpies
    dH_par = np.hstack((np.zeros(len(prod_names_con)), np.transpose(delta_H.drop('Reaction', axis = 1).to_numpy())[0]))
    
    # number of constant to find
    tar_names = set(targets.to_numpy()[0][1:])
    tar_num = [index for index, item in enumerate(prod_names) if item in tar_names]
    
    # number of enthalpy to find
    dH_names = np.hstack((prod_names_con, np.transpose(delta_H.drop('Value', axis = 1).to_numpy())[0]))
    un_el = set(prod_names) - set(dH_names)
    dH_ind_wtf = list([index for index, item in enumerate(prod_names) if item in un_el])
    un_el = set(prod_names) - (set(prod_names) - set(dH_names))
    dH_ind = list([index for index, item in enumerate(prod_names) if item in un_el])
    
    # creating array of diff total concentrations
    con_matrix_red = []
    for s in range(ser_num):
        for k in range(ser_counts[s] - 1):
            con_matrix_red.append(con_matrix[int(np.sum(ser_counts[:s])) + k + 1])
            
    if _calor_setup_type == 'DSC' or 'overfilled':
        _calor_setup_type_coeff = [[1] for i in range(len(con_matrix_red))]
    
    if _calor_setup_type == 'ampoule':
        _calor_setup_type_coeff = 1 / np.array(con_matrix_red)[:, idx] 
    
    return (ser_num, ser_counts, ser_info, volumes, heats, devs, tar_num, dH_ind, dH_ind_wtf, dH_par, 
            _calor_setup_type_coeff)

In [34]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np

def cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type):

    # finding the increment in equilibrium concentrations of products - needed for dH calculation
    vol_active = 0.988 # terrible solution. Gotta introduce new variable for operating volume 
    vol_init = 15      # of calorimetric cell for overfilled mode
    dif_conc = []        
             
    for s in range(ser_num):    
        for k in range(ser_counts[s] - 1):
            
            if _calor_setup_type == 'overfilled':
                factor = 1 - (volumes[int(np.sum(ser_counts[:s])) + k - s] / vol_active)
            else:
                factor = 1
            
            dif_conc.append(c_res_out[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out[int(np.sum(ser_counts[:s])) +
                                                                                       k] * factor)
            
    dif_conc, devs, volumes = np.array(dif_conc).astype(np.float32), devs.astype(np.float32), volumes.astype(np.float32)
    
    dif_vol = np.diff(volumes)
    dif_vol = np.array(np.hstack((volumes[0] - vol_init, dif_vol)))
    
    # finding vector of experimental values
    
    q_corr = deepcopy(heats)
    
    for i in range(len(heats)):
        for j in range(len(volumes[dH_ind])):
            
            if _calor_setup_type == 'overfilled':
                q_corr[i] -= vol_active * dif_conc[i, dH_ind[j]] * dH_par[j] 
            else:
                q_corr[i] -= (volumes[dH_ind[j]] * dif_conc[i, dH_ind[j]] * dH_par[j] +
                              dif_vol[i] * np.array(c_res_out)[i, dH_ind[j]] * dH_par[j])
    
    if _calor_setup_type == 'overfilled' or _calor_setup_type == 'DSC':
        c_eq = []
        for s in range(ser_num):
            for k in range(ser_counts[s] - 1):
                c_eq.append(c_res_out[int(np.sum(ser_counts[:s])) + k])
    else:
        c_eq = dif_conc
    
    if _calor_setup_type == 'overfilled':
        A = (vol_active * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff)
    
    else:
        A = ((np.array(volumes).reshape((len(volumes), 1)) * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff) +
             dif_vol.reshape((len(dif_vol), 1)) * np.array(c_eq)[:, dH_ind_wtf])
    
    AT = np.transpose(A)
    LTP = np.linalg.inv(np.dot(np.dot(AT, devs), A))
    RTP = np.dot(np.dot(AT, devs), q_corr)
    dH_res = np.dot(LTP, RTP)
    dH_res_tmp = dH_par

    for i in range(len(dH_ind_wtf)):
        dH_res_tmp = np.insert(dH_res_tmp, dH_ind_wtf[i], dH_res[i])

    dH_res_out = dH_res_tmp

    F_min, heats_calc = 0, []
    
    for i in range(len(heats)): 
        
        if _calor_setup_type == 'overfilled':
            F_min += (vol_active * np.sum(dH_res_out * dif_conc[i] * np.array(_calor_setup_type_coeff[i])) - heats[i]) ** 2
            
            heats_calc.append((np.sum(dH_res_out * dif_conc[i] * vol_active * _calor_setup_type_coeff[i])))
        
        else:
            F_min += ((np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i])) +
                       np.sum(dH_res_out * c_eq[i] * dif_vol[i]) - heats[i]) ** 2)
            
            heats_calc.append(np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i]))
                              + np.sum(dH_res_out * c_eq[i] * dif_vol[i]))
#     print('\neq con')
#     print(np.array(c_res_out))
#     print('\ndif con')
#     print(dif_conc)
#     print('\nvol')
#     print(volumes)
#     print('\ndH par')
#     print(dH_par)
#     print('\nsum of terms')
#     print(np.array(heats_calc))
#     print(heats)
    
    #print(sum(heats))
    #print(dH_res_out)
    #print(np.sum(dif_conc[i]))
    #print(np.sum(volumes[i]))
    #print(np.sum(np.array(_calor_setup_type_coeff[i])))
    
    # calculating st. dev. of dH_res / checking the quality of linear regression
    # consider making it in separate function
    
    disp = (F_min / (len(heats) - len(dH_ind_wtf)) * (np.diag(LTP))) ** 0.5  
    
    return F_min, dH_res_out, disp

In [47]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

from copy import deepcopy
import eq_evaluator
# import also script cl_enthalpies when needed

# main Hooke-Jeeves function --------------------------------

def cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type):

    # initial step
    
    basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type)
    
    basis_point_n = deepcopy(lg_k)
    
    print(new_min)

    # algorithm body
    
    for i in range(max_iter):
        
        print('\niteration', i, lg_k[tar_num])
        
        # check success of the previous step (i.e. estimation takes place
        # in the beginning of the iteration and not in the end)
        
        if F_min <= new_min:
        
            # fail ------------ #
        
            if step_init < HJ_eps:
                
                # exit 1: HS epsilon (lg_k move absolute threshold) achieved
                    
                print("p1 " + str(step_init) + " " + str(HJ_eps))
                break
            
            else:
                
                # reduce step and try exploratory move
                
                step_init /= 2
            
                basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
                print('\nF min more ', new_min, F_min, lg_k[tar_num])
                print(str(step_init))
        else:
            
            # success --------- #
            
            F_min = deepcopy(new_min)
            
            print('\nF min less', new_min, F_min, lg_k[tar_num])
            print(str(step_init))
            
            # first HS move
            
            basis_point_n = deepcopy(lg_k)
            lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])

            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                    con_matrix, ign_indices)

            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
            print('\nF min less after', new_min, F_min, lg_k[tar_num])
        
            # check success of the HS move
        
            if F_min <= new_min:
                lg_k = deepcopy(basis_point_n)
                print('\nconstant less after', basis_point[tar_num], lg_k[tar_num]) 
            else:
                F_min = new_min

            basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
        
#             # check success of the correcting exploratory move after HS move
#             # important! exploratory move takes place in the nearby of the point
#             # BEFORE or AFTER HS move depending on the fail or the success of the HS move
        
#             if F_min <= new_min:

#                # fail ------------ #

#                 print('\nF min p2', new_min, F_min, lg_k[tar_num])
#                 if step_init < HJ_eps:
#                     # exit 2: HS epsilon (lg_k move absolute threshold) achieved
#                     print("p2")
#                     break

#                 else: 
#                     # reduce step : what for???
#                     step_init /= 2

#             else:

#                 # success ------------ #
               
#                 F_min = deepcopy(new_min)
           
#                 print('\nF min p3.1', new_min, F_min, lg_k[tar_num])
           
#                 # second HS move
           
#                 basis_point_n = deepcopy(lg_k)
#                 lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])
           
#                 c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
#                                                        con_matrix, ign_indices)
   
#                 new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
#                                              , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
       
#                 # check success of the HS move
       
#                 if F_min <= new_min:
#                     lg_k = deepcopy(basis_point_n)
           
#                 print('\nF min p3.2', new_min, F_min, lg_k[tar_num])

    # calculate additional data to return

    c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                       con_matrix, ign_indices)
    new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                             , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)

    
    return F_min, lg_k, dH_res_out, c_res_out, disp


# exploratory move function used in main function ------------------------

def cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type):

    basis_point = deepcopy(lg_k)
    
    c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                con_matrix, ign_indices)

    new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)

    #if F_min > new_min:
    F_min = new_min
    
    for j in range(len(tar_num)): 
        #pdb.set_trace()
        print('input', lg_k[tar_num[j]], F_min)
        
        # move right ---------------- #
        
        lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
        
        # equilibrium
        c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)
        
        # enthalpies
        new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes
                                                  , heats, dH_ind, dH_par, dH_ind_wtf
                                                  , _calor_setup_type_coeff, _calor_setup_type)
        
        print('right', lg_k[tar_num[j]], new_min, F_min)
        
        if F_min <= new_min:
            
            # move left ------------- #
            
            lg_k[tar_num[j]] = lg_k[tar_num[j]] - 2 * step_init
            
            # equilibrium
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                        con_matrix, ign_indices)
        
            # enthalpies
            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                                      , dH_ind, dH_par, dH_ind_wtf
                                                      , _calor_setup_type_coeff, _calor_setup_type)
            
            print('left', lg_k[tar_num[j]], new_min, F_min)
            
            # restore initial value - #
            # if move failed -------- #
            
            if F_min <= new_min:
                
                lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
                new_min = deepcopy(F_min)
                
    print('return', basis_point[tar_num[j]], lg_k[tar_num[j]])
    
    return basis_point, lg_k, new_min, c_res_out, disp

In [48]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------
import numpy as np
import eq_evaluator
# import also script cl_enthalpies when needed

def cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                      ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                      _calor_setup_type_coeff, _calor_setup_type):
    
    # creating constants vectors ---------------------------------
    lg_k_1, lg_k_2 = np.array(lg_k), np.array(lg_k)
    heats_one, heats_two = [0] * len(heats), [0] * len(heats)
    jc_mtrx = [[0] * len(heats) for i in range(len(tar_num))]

    for i in range(len(tar_num)):
        # calculating the heats for Jacobi matrix computing
        lg_k_1[tar_num[i]] += HJ_eps
        lg_k_2[tar_num[i]] -= HJ_eps
    
        # if lg_k is overestimated
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_1,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])
    
        for o in range(len(heats)):
            heats_one[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
        # if lg_k is underestimated    
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_2,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])    
   
        for o in range(len(heats)):
            heats_two[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
        # calculating the Jacobi matrix    
        jc_mtrx[i] = (np.array(heats_one) - np.array(heats_two)) / (2 * HJ_eps)

        jc_mtrx, devs = np.array(jc_mtrx).astype(np.float32), devs.astype(np.float32)

    cov_matrix = (F_min / (len(heats) - len(tar_num))) * np.linalg.inv(np.dot(np.dot(jc_mtrx, devs), np.transpose(jc_mtrx)))

    errs_values = np.diag(cov_matrix) ** 0.5
    return errs_values

In [55]:
# input for xlsx file

_subdir = "calorimetry"
_sep = ";"
_file = "test_6.xlsx"

# use 'DSC' for test_1 test_2 and test_5
#     'ampoule' for test_3
#     'overfilled' for test_4
_calor_setup_type = 'DSC' 

# algo parameters
max_iter, eps, HJ_eps, step_init = 100, 0.0000001, 0.000001, 0.5


# load data

st_coeff_data, lg_k_data, con_data, type_con, component_name_for_yields = eq_data.eq_scripts_load(_sep, _subdir, _file)
heats_data, targets, delta_H = cl_scripts_load(_sep, _subdir, _file)

# preproc

st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, idx, type_con = eq_preproc.eq_preproc(
    st_coeff_data, con_data, type_con, lg_k_data, component_name_for_yields)

(ser_num, ser_counts, ser_info, volumes, heats, devs, 
 tar_num, dH_ind, dH_ind_wtf, dH_par, _calor_setup_type_coeff) = cl_preproc(
    con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type)

# calculation of eq concentration (starting point)
c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)

# calculation of F_min and unknown enthalpies (starting point)
F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type)

# F_min assignment for further calculations
new_min = F_min

# constant refinment
if len(tar_num) > 0:
    F_min, lg_k, dH_res_out, c_res_out, disp = cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type)


F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type)

# error estimation
if len(tar_num) > 0:
    errs_values = cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                                    ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                                    _calor_setup_type_coeff, _calor_setup_type)

input [2.] 0.001075556070945117
right [2.5] 0.0006147661725648193 0.001075556070945117
input [3.] 0.001075556070945117
right [3.5] 0.0006032502042558203 0.001075556070945117
return [3.] [3.5]
0.0006032502042558203

iteration 0 [[2.5]
 [3.5]]

F min less 0.0006032502042558203 0.0006032502042558203 [[2.5]
 [3.5]]
0.5

F min less after 0.00018475565482382682 0.0006032502042558203 [[3.]
 [4.]]
input [3.] 0.00018475565482382682
right [3.5] 1.4325776639181638e-06 0.00018475565482382682
input [4.] 0.00018475565482382682
right [4.5] 4.926857556828813e-07 0.00018475565482382682
return [4.] [4.5]

iteration 1 [[3.5]
 [4.5]]

F min less 4.926857556828813e-07 4.926857556828813e-07 [[3.5]
 [4.5]]
0.5

F min less after 0.00021605780985695566 4.926857556828813e-07 [[4.]
 [5.]]

constant less after [[3.]
 [4.]] [[3.5]
 [4.5]]
input [3.5] 4.926857556828813e-07
right [4.] 0.00023513766101287384 4.926857556828813e-07
left [3.] 0.000204804796972407 4.926857556828813e-07
input [4.5] 4.926857556828813e-07
r

In [7]:
print(np.array(c_res_out))
#print(c_res_out[int(np.sum(ser_counts[:0])) + 1])

[[2.39381965e-17 1.00000000e-03 7.60618042e-17 3.86837491e-32]
 [5.64842374e-05 3.38508086e-03 6.07535293e-04 7.29070102e-07]
 [1.33786787e-04 2.79513196e-03 1.18820325e-03 3.37733756e-06]
 [2.42745449e-04 2.24195334e-03 1.72923040e-03 8.91814369e-06]
 [3.99373570e-04 1.74296271e-03 2.21178349e-03 1.87668956e-05]
 [6.23262314e-04 1.31941614e-03 2.61293049e-03 3.45994529e-05]
 [9.29314965e-04 9.87353310e-04 2.91548434e-03 5.75630970e-05]
 [1.31691730e-03 7.45806333e-04 3.12075565e-03 8.73149837e-05]
 [1.77037135e-03 5.77371619e-04 3.24784279e-03 1.22160241e-04]
 [2.26996369e-03 4.60358925e-04 3.32040204e-03 1.60132723e-04]
 [2.79984524e-03 3.77386054e-04 3.35733802e-03 1.99709904e-04]
 [3.34934953e-03 3.16783996e-04 3.37131164e-03 2.39899797e-04]
 [3.91170305e-03 2.71162690e-04 3.37031899e-03 2.80096314e-04]
 [4.48261755e-03 2.35859375e-04 3.35938692e-03 3.19935345e-04]
 [5.05932348e-03 2.07875772e-04 3.34173124e-03 3.59198468e-04]
 [5.63998115e-03 1.85231587e-04 3.31946349e-03 3.977553

In [24]:
#print('\nStoich coeff data')
#print(st_coeff_data)

#print('\nlg K')
#print(lg_k_data)

#print('\nconcentrations')
#print(con_data)

#print('\ntype con')
#print(type_con)

#print('\nexperimental data')
#print(heats_data)

print('\n_calor_setup_type_coeff')
print(_calor_setup_type_coeff)

print('\ntargets')
print(targets)

#print('\ndelta_H')
#print(delta_H)


_calor_setup_type_coeff
[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]

targets
            0   1    2
0  constants   HL  H2L


In [41]:
print('\nSeries')
print(ser_num)
print(ser_counts)
print(ser_info)

print('\nSt coeff matr')
print(st_coeff_matrix)

print('\nprod names')
print(prod_names)
print(prod_names_con)

print('\nconst')
print(lg_k)

print('\nconcentrations')
print(con_matrix)

print(ign_indices)

print('\ntype con')
print(type_con)

print('\nvolumes')
print(volumes)

print('\nheats')
print(heats)

print('\ndeviations')
print(devs)

print('\nwhat to find and what is known')
print(tar_num)
print(dH_ind)
print(dH_ind_wtf)

print('\nenthalpies')
print(dH_par)


Series
1
[21]
['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']

St coeff matr
[[1 0]
 [0 1]
 [1 1]
 [2 1]]

prod names
['H', 'L', 'HL', 'H2L']
['H', 'L']

const
[[0.        ]
 [0.        ]
 [3.50671387]
 [4.87670898]]

concentrations
[[1.00000000e-16 1.00000000e-03]
 [6.65477670e-04 3.99334522e-03]
 [1.32874472e-03 3.98671255e-03]
 [1.98981213e-03 3.98010188e-03]
 [2.64869085e-03 3.97351309e-03]
 [3.30539171e-03 3.96694608e-03]
 [3.95992550e-03 3.96040075e-03]
 [4.61230292e-03 3.95387697e-03]
 [5.26253463e-03 3.94737465e-03]
 [5.91063117e-03 3.94089369e-03]
 [6.55660306e-03 3.93443397e-03]
 [7.20046073e-03 3.92799539e-03]
 [7.84221453e-03 3.92157785e-03]
 [8.48187477e-03 3.91518125e-03]
 [9.11945166e-03 3.90880548e-03]
 [9.75495538e-03 3.90245045e-03]
 [1.03883960e-02 3.89611604e-03]
 [1.10197836e-02 3.88980216e-03]
 [1.16491281e-02 3.88350872e-03]
 [1.22764394e-02 3.87723561e-03]
 [1.29017274e-02 3.87098273e-03]]
[]

type con
0    tot
1    tot
Name: 0, dtype: object



In [26]:
print('\nEq conc')
print(np.array(c_res_out))
print(np.shape(c_res_out))


Eq conc
[[2.37442616e-17 1.00000000e-03 7.62557392e-17 4.24927269e-32]
 [5.59304180e-05 3.38459597e-03 6.07951262e-04 7.97995105e-07]
 [1.32485794e-04 2.79415009e-03 1.18886601e-03 3.69645668e-06]
 [2.40415683e-04 2.24046568e-03 1.72987594e-03 9.76025472e-06]
 [3.95619065e-04 1.74097869e-03 2.21199701e-03 2.05373851e-05]
 [6.17578604e-04 1.31699160e-03 2.61209587e-03 3.78586176e-05]
 [9.21163161e-04 9.84608446e-04 2.91282226e-03 6.29700365e-05]
 [1.30583945e-03 7.42891218e-04 3.11550803e-03 9.54777238e-05]
 [1.75606191e-03 5.74406566e-04 3.23946346e-03 1.33504624e-04]
 [2.25226882e-03 4.57414273e-04 3.30859649e-03 1.74882933e-04]
 [2.77872555e-03 3.74495675e-04 3.34199910e-03 2.17939214e-04]
 [3.32484046e-03 3.13962061e-04 3.35244645e-03 2.61586940e-04]
 [3.88388533e-03 2.68413920e-04 3.34799886e-03 3.05165265e-04]
 [4.45159797e-03 2.33184128e-04 3.33371746e-03 3.48279669e-04]
 [5.02522288e-03 2.05272552e-04 3.31283708e-03 3.90695856e-04]
 [5.60292704e-03 1.82698014e-04 3.28747653e-03

In [56]:
print('\nlg K')
print(lg_k)

print('\nF_min')
print(F_min)

print('\nFinal enthalpies')
print(dH_res_out)

print('\ncalculated error of enthalpy')
print(disp)

if len(tar_num) > 0:
    print('\ncalculated error of constant')
    print(errs_values)


lg K
[[0.    ]
 [0.    ]
 [3.5   ]
 [4.8125]]

F_min
6.233193332483898e-09

Final enthalpies
[0.0 0.0 9.9961273835174 12.945013411992601]

calculated error of enthalpy
[0.0010031  0.00878524]

calculated error of constant
[0.00149698 0.0016604 ]


In [12]:
#import matplotlib.pyplot

#s = 4.5

#xxx = []
#FFF = []
#while s < 5.5:
    #lg_k[tar_num] = s
    
    #c_res_out = eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
        #con_matrix, ign_indices, ser_counts, ser_info)
        
    #F_min, dH_res_out = enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff)
    
    #xxx.append(s)
    #FFF.append(F_min)
    #s += 0.02


#matplotlib.pyplot.plot(xxx, FFF, 'ro-')

#import sys
#import os
##sys.path.append('../kev_calorim')
#print(os.path.abspath('../kev/'))
#print(os.getcwd())

#lg_k is array

In [13]:
import matplotlib.pyplot as plt
from copy import copy, deepcopy

def risovalka(max_iter, eps, st_coeff_matrix, type_con, lg_k, con_matrix, ign_indices, ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type):

    s = [[0] * 42 for i in range(42)]
    const_x = [0] * 42
    const_y = [0] * 42
    s = np.array(s)

    i, a = 2.5, 0

    for counter_x in range(len(s)):
        j = 3.3
        const_x[counter_x] = round(i, 2)
        for counter_y in range(len(s)):
            lg_k[2], lg_k[3] = np.array(i), np.array(j)
            const_y[counter_y] = round(j, 2)
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                            con_matrix, ign_indices)
        
            F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
            
            s[counter_x, counter_y] = float(F_min * 100000000)
        
            j += 0.05
    
        i += 0.05

    s = pd.DataFrame(data = s, index = const_y, columns = const_x)

    fig, ax = plt.subplots(figsize=(20, 20))
    im = ax.imshow(s, cmap = 'RdYlGn_r')
    ax.set_xticks(np.arange(len(const_x)))
    ax.set_yticks(np.arange(len(const_y)))

    ax.set_xlabel('lg K1')
    ax.set_ylabel('lg K2')

    ax.set_xticklabels(const_x)
    ax.set_yticklabels(const_y)

    plt.show()
    return s

In [14]:
#comment this section for reducing the waiting time
#s = risovalka(max_iter, eps, st_coeff_matrix, type_con, lg_k, con_matrix, ign_indices, ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff, _calor_setup_type)
#with pd.ExcelWriter('d:/KEVreplica/output/calorimetry/test_6.xlsx', mode = "w") as output:
    #s.to_excel(output)

In [15]:
F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type) 


eq con
[[2.39381965e-17 1.00000000e-03 7.60618042e-17 3.86837491e-32]
 [5.64842374e-05 3.38508086e-03 6.07535293e-04 7.29070102e-07]
 [1.33786787e-04 2.79513196e-03 1.18820325e-03 3.37733756e-06]
 [2.42745449e-04 2.24195334e-03 1.72923040e-03 8.91814369e-06]
 [3.99373570e-04 1.74296271e-03 2.21178349e-03 1.87668956e-05]
 [6.23262314e-04 1.31941614e-03 2.61293049e-03 3.45994529e-05]
 [9.29314965e-04 9.87353310e-04 2.91548434e-03 5.75630970e-05]
 [1.31691730e-03 7.45806333e-04 3.12075565e-03 8.73149837e-05]
 [1.77037135e-03 5.77371619e-04 3.24784279e-03 1.22160241e-04]
 [2.26996369e-03 4.60358925e-04 3.32040204e-03 1.60132723e-04]
 [2.79984524e-03 3.77386054e-04 3.35733802e-03 1.99709904e-04]
 [3.34934953e-03 3.16783996e-04 3.37131164e-03 2.39899797e-04]
 [3.91170305e-03 2.71162690e-04 3.37031899e-03 2.80096314e-04]
 [4.48261755e-03 2.35859375e-04 3.35938692e-03 3.19935345e-04]
 [5.05932348e-03 2.07875772e-04 3.34173124e-03 3.59198468e-04]
 [5.63998115e-03 1.85231587e-04 3.31946349e-03 